In [240]:
import os
from pathlib import Path

import gffutils
import pandas as pd
import sqlite3

in_dir = "input"
out_dir = "output"
Path(out_dir).mkdir(exist_ok=True)

## Create GFF databases and read in features

In [242]:
def create_db(path, db_filename, **kwargs):
    """
    Create/connect db for given gff file path
    """
    if not os.path.isfile(os.path.join(out_dir, db_filename)):
        db = gffutils.create_db(path, os.path.join(out_dir, db_filename), force=True, **kwargs)
    else:
        db = sqlite3.connect(os.path.join(out_dir, db_filename), check_same_thread=False)
        db = gffutils.FeatureDB(db)
    return db

# WormBase release WS283 canonical three_prime_UTR for C. elegans chromosome I
control_fn = os.path.join(in_dir, "WormBase.I.three_prime_UTR.gff3")
db_control = create_db(control_fn, "control.db")
control = list(db_control.all_features())

# Output from UTRme run
test = os.path.join(in_dir, "utrme_output.gff3")
db_test = create_db(test, "test_utrme.db")
test = list(db_test.all_features())

# WormBase release WS283 canonical mRNA transcripts for C. elegans chromosome I
genes_fn = os.path.join(in_dir, "WormBase.I.mRNA.gff3")
db_genes = create_db(genes_fn, "genes.db")
genes = list(db_genes.all_features())

## Prepare test dataset

C. elegans contains alternative mRNA transcripts for each gene, and the UTRme UTRs refer only to these mRNA transcript "names" in their attributes. Therefore, it will be helpful to create a sequence name -> gene id mapping dataframe to allow comparisions between the two datasets.

In [4]:
gdf = pd.DataFrame([vars(c) for c in genes], columns=["featuretype", "start", "end", "strand", "attributes"])
gdf = gdf[gdf["featuretype"] == "mRNA"]
gdf["gene"] = gdf["attributes"].apply(lambda x: x.get("Parent")[0].split(":")[-1])
gdf["name"] = gdf["attributes"].apply(lambda x: x.get("Name")[0])
gdf = gdf[["name", "gene"]]
gdf

,name,gene
0,Y74C9A.3.1,WBGene00022277
1,Y74C9A.2a.3,WBGene00022276
2,Y74C9A.2a.1,WBGene00022276
3,Y74C9A.2a.2,WBGene00022276
4,Y74C9A.2b.1,WBGene00022276
...,...,...
4698,F31C3.1.1,WBGene00000881
4699,F31C3.3.1,WBGene00009285
4700,F31C3.4.1,WBGene00009286
4701,F31C3.5.1,WBGene00009287


Create a dataframe of UTRme test UTRs with a "gene" column consisting of the parent gene's id.

In [35]:
tdf = pd.DataFrame([vars(t) for t in test], columns=["start", "end", "strand", "attributes"])
tdf["gene"] = tdf["attributes"].apply(lambda x: x.get("ID")[0])
tdf = pd.merge(tdf, gdf, left_on="gene", right_on="name")
tdf = tdf.rename({'gene_y': "gene"}, axis=1).drop(["gene_x", "attributes"], axis=1).set_index("gene")
tdf

,start,end,strand,name,gene
0,22939,24650,-,Y74C9A.4a.1,WBGene00022278
1,80344,81000,+,Y48G1C.2b.1,WBGene00000812
2,101245,101504,-,Y48G1C.8a.1,WBGene00021681
3,108769,109264,-,F53G12.1.1,WBGene00004274
4,133582,134271,+,F53G12.5b.1,WBGene00003229
...,...,...,...,...,...
502,14823733,14824009,+,Y54E5B.4.1,WBGene00006711
503,14833448,14834697,-,F32A7.3a.1,WBGene00009304
504,14951567,14952030,+,ZK909.2h.1,WBGene00002189
505,14965021,14965417,+,ZK909.4.1,WBGene00000469


Aggregate transcripts for minimum start base and maximum end base to capture full extent of the UTR for each gene.

In [233]:
test_utrs = tdf.groupby(["gene", "strand"]).agg({"start": min, "end": max}).reset_index("strand")

# Forward strand UTR start bases match those of canonical UTRs minus 1
test_utrs.loc[test_utrs["strand"]=="+", "start"] += 1
test_utrs

,strand,start,end
gene,,,
WBGene00000010,-,11388803,11389140
WBGene00000084,+,7608007,7609295
WBGene00000148,+,10345475,10345594
WBGene00000150,-,5342174,5342480
WBGene00000156,+,8055142,8055482
...,...,...,...
WBGene00219288,-,13526268,13527384
WBGene00219313,+,9416506,9416572
WBGene00235158,-,2148830,2150194


## Prepare control dataset

Create a dataframe of canonical WormBase control UTRs with a "name" column consisting of the parent gene's name.

In [130]:
cdf = pd.DataFrame([vars(c) for c in control], columns=["start", "end", "source", "strand", "attributes"])
cdf = cdf[cdf.source == "WormBase"]
cdf['name'] = cdf["attributes"].apply(lambda x: x.get("Parent")[0].split(":")[-1])
cdf = cdf.drop(['attributes', "source"], axis=1)
cdf = pd.merge(cdf, gdf, on="name")
cdf

,start,end,strand,name,gene
0,4116,4220,-,Y74C9A.3.1,WBGene00022277
1,16586,16833,+,Y74C9A.2a.2,WBGene00022276
2,16586,16837,+,Y74C9A.2a.1,WBGene00022276
3,16702,16793,+,Y74C9A.2a.3,WBGene00022276
4,17484,17910,-,Y74C9A.4a.1,WBGene00022278
...,...,...,...,...,...
4192,15040474,15040608,-,F31C3.1.1,WBGene00000881
4193,15041433,15041884,-,F31C3.3.1,WBGene00009285
4194,15050797,15051145,-,F31C3.4.1,WBGene00009286
4195,15053608,15053793,-,F31C3.6a.1,WBGene00009288


Aggregate transcripts for minimum start base and maximum end base to capture full extent of the UTR for each gene.

In [226]:
control_utrs = cdf.groupby(["gene", "strand"]).agg({"start": min, "end": max}).reset_index("strand")
control_utrs

,strand,start,end
gene,,,
WBGene00000001,+,5109948,5110183
WBGene00000006,-,2571740,2571987
WBGene00000010,-,11388682,11389140
WBGene00000012,-,3886070,3886110
WBGene00000020,-,2255921,2256151
...,...,...,...
WBGene00304996,+,5341995,5342045
WBGene00305018,-,9588662,9588719
WBGene00306080,-,2939477,2939518


# Compare the two datasets

Now that we have two comparable datasets, we can calculate some statistical differences. We determine length of each UTR in both dataframes, and determine differences of test UTR length to true UTR length.

In [236]:
control_utrs["len"] = abs(control_utrs["end"] - control_utrs["start"])
test_utrs["len"] = abs(test_utrs["end"] - test_utrs["start"])
test_utrs["difference"] = (test_utrs["len"] - control_utrs["len"])
test_utrs

,strand,start,end,len,difference
gene,,,,,
WBGene00000010,-,11388803,11389140,337,-121.0
WBGene00000084,+,7608007,7609295,1288,794.0
WBGene00000148,+,10345475,10345594,119,-60.0
WBGene00000150,-,5342174,5342480,306,6.0
WBGene00000156,+,8055142,8055482,340,7.0
...,...,...,...,...,...
WBGene00219288,-,13526268,13527384,1116,1098.0
WBGene00219313,+,9416506,9416572,66,-131.0
WBGene00235158,-,2148830,2150194,1364,995.0


## Supplementary Table 2 statistics

In [238]:
total_test = len(test_utrs)
total_control = len(control_utrs)

genes_missed_utr = control_utrs.index.difference(test_utrs.index)
num_missed_utrs = len(genes_missed_utr)
genes_no_utr = set(gdf["gene"]).difference(control_utrs.index)
novel_utr_genes = genes_no_utr.intersection(test_utrs.index)
num_novel_utrs = len(novel_utr_genes)

BP1 = 50
BP2 = 200
num_matched =  len(test_utrs[abs(test_utrs["difference"]) <= BP1])
num_reduced = len(test_utrs[test_utrs['difference'] < -BP1])
num_extended1 = len(test_utrs[(test_utrs['difference'] > BP1) & (test_utrs['difference'] <= BP2)])
num_extended2 = len(test_utrs[test_utrs['difference'] > BP2])

print("Canonical utrs that are missed by UTRme: {} ({:.2f}%)".format(num_missed_utrs, 100*num_missed_utrs/total_control))
print("UTRme utrs that match true utr to within {} bases: {} ({:.2f}%)".format(BP1, num_matched, 100*num_matched/total_control))
print("UTRme utrs that reduce true utr by >{} bases: {} ({:.2f}%)".format(BP1, num_reduced, 100*num_reduced/total_control))
print("UTRme utrs that extend true utr by between {} and {} bases: {} ({:.2f}%)".format(BP1, BP2, num_extended1, 100*num_extended1/total_control))
print("UTRme utrs that extend true utr by >{} bases: {} ({:.2f}%)".format(BP2, num_extended2, 100*num_extended2/total_control))
print("UTRme utrs called where no true utr existed previously: {}".format(num_novel_utrs))
print("Total UTRme UTRs: {}".format(total_test))

Canonical utrs that are missed by UTRme: 2119 (81.72%)
UTRme utrs that match true utr to within 50 bases: 157 (6.05%)
UTRme utrs that reduce true utr by >50 bases: 121 (4.67%)
UTRme utrs that extend true utr by between 50 and 200 bases: 19 (0.73%)
UTRme utrs that extend true utr by >200 bases: 177 (6.83%)
UTRme utrs called where no true utr existed previously: 22
Total UTRme UTRs: 496


Save list of gene ids with novel 3' UTRs called by UTRme

In [239]:
with open(os.path.join(out_dir, "novel_utr_genes_utrme.txt"), "w") as f:
    for g in sorted(list(novel_utr_genes)):
        print(g)
        f.write(g + "\n")

WBGene00000632
WBGene00001745
WBGene00003977
WBGene00006044
WBGene00007976
WBGene00010291
WBGene00011651
WBGene00011934
WBGene00012389
WBGene00012483
WBGene00013404
WBGene00016727
WBGene00017908
WBGene00018596
WBGene00019992
WBGene00020556
WBGene00021473
WBGene00022112
WBGene00022367
WBGene00044348
WBGene00044435
WBGene00305050
